# TP Réseaux de Neurones – Séance 2
## Réseau à une couche cachée (MLP simple)

### Objectifs
- Comprendre pourquoi un perceptron simple ne peut pas résoudre le problème du XOR.
- Implémenter un réseau à une couche cachée.
- Propagation avant et rétropropagation.
- Visualiser la frontière de décision.


## 1. Le problème du XOR
Le jeu de données du "ou exclusif" est simple. La sortie $y=1$ si un et un seul des $x_i$ vaut 1 et $y=0$ dans les autres cas. Un problème simple mais qui n'est pas linéairement séparable lorsqu'on voit la disposition des 4 points :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])

plt.scatter(X[:,0], X[:,1], c=y, cmap="bwr", s=100)
plt.xlabel("x1"); plt.ylabel("x2")
plt.title("Problème du XOR")
plt.show()

Le perceptron de base ne peut donc pas résoudre le problème de modélisation posé par la fonction XOR. Cela a constitué un échec cinglant dans les années 60 pour les chercheurs travaillant dans le champ des réseaux de neurones (voir livre "Perceptron" de Minsky & Papert (1969) qui détaille les limites du perceptron). Mais ce que l'on n'avait pas compris encore à l'époque (surtout dans l'industrie) c'est qu'en associant plusieurs neurones (comme c'est le cas dans le cerveau animal) on pouvait résoudre le problème du XOR, ce qu'on appellera plus tard le perceptron multicouche. Cet échec public du premier perceptron a découragé les financements pendant une décénie (que l'on nommera période de glaciation pour l'IA, 1974-1980). Quelques travaux ont tout de même continué et ce n'est qu'au milieu des années 80 que les réseaux de neurones ont connu une seconde vague d'intérêt. 

## 2. Le perceptron multicouche ou réseau feed-forward multicouche
Le principe est simple. Il s'agit de disposer des neurones en couches composées de 1 ou plusieurs neurones chacune :
- une couche d'entrée qui reçoit les données d'entrée. Le nombre de neurones y est égal au nombre de composantes du vecteur d'entrée. Dans notre réseau feed-forward chaque neurone de la couche d'entrée est relié à tous les neurones de la 1ère couche caché qui suit. Mais il existe d'autres architectures avec des connexions manquantes.
- une ou plusieurs couches dites cachées dont les neurones reçoivent en entrée les sorties des neurones de la couche précédente et sont reliés aux neurones de la couche suivante. Ce sont ces couches cachées qui vont permettre de traiter des problèmes non linéaires.
- une couche de sortie dont les neurones reçoivent les sorties de la dernière couche cachée et qui fournissent en sortie la réponse/prédiction de notre modèle.
![réseau feed forward](https://raw.githubusercontent.com/thierrycondamines/enseignements/main/NSI/Neurones/reseau_neurones.png)
Lorsque les données sont entrées dans la couche d'entrée, elles sont envoyées aux neurones de la première couche cachée. Ces neurones vont pondérer (chacun avec des poids et biais spécifiques) ces entrées et produire combinaison linéaire, comme on l'a vu avec le perceptron simple, qui sera passée à la fonction d'activation (souvent à valeurs dans [0,1] ou [-1,1]). Lorsque la valeur sortant d'un neurone est non nulle, on dit qu'elle active le neurone suivant qui la reçoit. Cette propagation vers l'avant dans le réseau a donne son nom de réseau feed-forward.

Les neurones d'une même couche peuvent avoir la même fonction d'activation (ce sera notre cas ici) mais ce n'est pas obligatoire. La couche de sortie utilise généralement une fonction d'activation de type sygmoïde ou $softmax$.

Pour résoudre le problème du XOR, un réseau 2-2-1 suffit (2 neurones d'entrée, 2 neurones cachés, 1 neurone de sortie). La couche d'entrée ne traîte pas les entrées (pas de poids ni fonction d'activation) et ne fait que les distribuer sur la couche cachée. Pour les fonctions d'activation, nous allons utiliser : 
- tanh pour les 2 neurones de la couche cachée
- sigmoïde pour le neurone de sortie

Tanh est ici plus adaptée pour plusieurs raisons :
- elle est centrée sur 0 (intervalle [-1,1]) alors que sigmoïde est décalée positive ([0,1]) ce qui évite à la mise à jour des poids de zigzaguer et donc une convergence plus rapide et plus stable;
- la pente est plus forte avec tanh autour de 0, ce qui rend l'apprentissage plus "vif" (gradient plus grand);
- tanh permet de coder la présence ou l'absence d'une caractéristique par des valeurs négatives (absence) ou positives (présence) ce qui est souvent utile;
- Même si on peut aproximer XOR avec sigmoïde, l'optimisation est en général plus lente et instable.

La fonction sigmoïde, bien qu'historique, est beacoup moins utilisée de nos jours. On la conserve juste dans le neurone de sortie car on veut des sorties 0 ou 1.

## 3. Initialisation des paramètres
Chaque neurone du réseau a ses poids qui amplifient ou atténuent un signal. Plus un poids est élevé plus l'entrée associée affectera le résultat de sortie. C'est en faisant évoluer ces poids dans la phase d'apprentissage que le réseau va apprendre à l'aveugle les corrélations qui existent entre les données d'entrées et leur caractéristiques (classes) données en sortie.
Nous conservons les biais, introduits pour le perceptron simple. Ceux-ci, dans un réseau multicouche, vont garantir l'activation d'au moins quelques neurones par couches même dans le cas d'un signal faible (proche de zéro). Conceptuellement, Ils permettent ainsi au réseau de tester de nouvelles interprétations.

Les poids et biais seront à nouveau initialisés aléatoirement mais avec une petite variante : l'initialisation dite de Xavier/Glorot. Cette initialisation, proposée par Xavier Glorot et Joshua Bengio en 2010, propose de prendre les poids dans une loi uniforme $U[-\sqrt {\frac{6}{nbin+nbout}},\sqrt {\frac{6}{nbin+nbout}}]$, ce qui a pour effet de stabiliser les activations et les gradients dès le départ. Ici, pour la couche cachée $nbin=2$ et $nbout=2$ et pour la couche de sortie $nbin=2$ et $nbout=1$

Afin de mieux voir les biais, j'ai choisi de ne pas les intégrer aux vecteurs de poids mais cela reste bien entendu tout à fait possible.

In [ ]:
np.random.seed(0)
# Poids des deux neurones de la couche cachée
a = np.sqrt(6/4)
w1 = np.random.uniform(-a, a)
w2 = np.random.uniform(-a, a)
w3 = np.random.uniform(-a, a)
w4 = np.random.uniform(-a, a)

# Biais des 2 neurones de la couche cachée
b1 = 0.
b2 = 0.

# poids du neurone de sortie
a=np.sqrt(6/3)
w5 = np.random.uniform(-a, a)
w6 = np.random.uniform(-a, a)

# Biais du neurone de sortie
b = 0.



## 4. Fonction sigmoide

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

## 5. Fonction de coût

In [ ]:
def perte(y_true, y_pred):
    eps = 1e-9
    return -np.mean(y_true*np.log(y_pred+eps) + (1-y_true)*np.log(1-y_pred+eps))

## 6. Rétropropagation
L'apprentissage par rétropropagation a été inventé en 1969 par Bryson et Ho mais est resté dans les placards jusque dans le milieu des années 80 où on s'est apperçu de son importance dans la réduction de l'erreur.

Nous avons vu, pour un perceptron simple, que, lorsque la sortie correspond à l'étiquette (valeur exacte), on ne change rien et, lorsqu'elle diffère, on ajuste les poids et le biais. La rétropropagation est une généralisation de ce principe à un réseau de plusieurs neurones. Mais comme il y a une multitude de poids dans un réseau de neurones, il s'agit de répartir/diviser la responsabilité de l'erreur entre les poids.

Pour un perceptron simple, il n'y a qu'un seul poids par entrée qui influence la sortie. Par contre dans un réseau de neurone, une entrée est reliée à la sortie via une multitude de poids ce qui rend le problème de modification des poids plus complexe. Comme le seul élément qui nous guide est l'erreur en sortie du réseau, on va donc partir de la couche sortie et faire en remontant vers la couche d'entrée (d'où le nom de rétropropagation).

Pour bien visualiser le mécanisme de rétropropagation, voici un schéma du réseau où apparaissent séparément chaques paramètres :

![perceptron_2_2_1](https://raw.githubusercontent.com/thierrycondamines/enseignements/main/NSI/Neurones/perceptron_2_2_1.png)

Pour rappel, en prenant une fonction coût de la forme $E(W)=-[y.log(\hat y)+(1-y).log(1-\hat y)]$, nous avions défini un paramètre $\delta$ :$$\delta = \frac{\partial E}{\partial v} = \hat y - y$$

### 6.1. Couche de sortie

La couche de sortie est composée d'un seul perceptron de poids $w_5$ et $w_6$, donnant une combilaison linéaire de ses entrées $y_1$ et $y_2$ venant des neurones de la couche cachée : $$v = w_5.y_1+w_6.y_2+b$$. Cette combinaison est ensuite passée à la fonction sigmoïde pour donner la sortie finale $\hat y = s(v)$. On en déduite les dérivées partielles :
$$\frac{\partial E}{\partial w_5} = \frac{\partial E}{\partial v}.\frac{\partial v}{\partial w_5} = \delta.y_1 \implies \Delta w_5 = -\eta \frac{\partial E}{\partial w_5} = -\eta.\delta.y_1$$
et de la même façon :$$\frac{\partial E}{\partial w_6} =\delta.y_2 \implies \Delta w_6 =  -\eta.\delta.y_2$$
$$\frac{\partial E}{\partial b} =\delta \implies \Delta b =  -\eta.\delta$$

### 6.2. Couche cachée

Il s'agit ici de voir l'impact des poids $w_1, ..., w_4$, sur l'erreur de sortie. Les deux neurones de cette couche cachée on une fonction d'activation de type $\phi(x)=tanh(x)$ ce qui nous donne $y_i = \phi (v_i) = tanh(v_i)$ et par suite $$\frac{\partial y_i}{\partial v_i} = 1 - tanh^2 (v_i) = 1 - y_i^2$$
Passons maintenant aux dérivées partielles par rapport aux poids de cette couche :
$$\frac{\partial E}{\partial w_1} = \frac{\partial E}{\partial v}.\frac{\partial v}{\partial y_1}.\frac{\partial y_1}{\partial v_1}.\frac{\partial v_1}{\partial w_1} = \delta.w_5.(1-y_1^2).x_1$$
Remarquons ici que le calcul de cette dérivée se fait en partant de la sortie et en remontant le long des branches du réseau jusqu'à $w_1$. Comme on l'avait fait pour le paramètre $\delta$, on peut poser $\delta_1 = \delta.w_5.(1-y_1^2)$ ce qui nous donne :
$$\frac{\partial E}{\partial w_1} = \delta_1 . x_1 \implies \Delta w_1 =  -\eta.\delta_1.x_1$$
et de la même manière on obtient :
$$\frac{\partial E}{\partial w_3} =\delta.w_5.(1-y_1^2).x_2 \implies \Delta w_3 =  -\eta.\delta_1.x_2$$
$$\frac{\partial E}{\partial b_1} =\delta_1 \implies \Delta b_1 =  -\eta.\delta_1$$
et, en posant $\delta_2 = \delta.w_6.(1-y_2^2)$ :
$$\frac{\partial E}{\partial w_2} =\delta_2.x_1 \implies \Delta w_2 =  -\eta.\delta_2.x_1$$
$$\frac{\partial E}{\partial w_4} =\delta_2.x_2 \implies \Delta w_4 =  -\eta.\delta_2.x_2$$
$$\frac{\partial E}{\partial b_2} =\delta_2 \implies \Delta b_2 =  -\eta.\delta_2$$

### 6.3. Traitement de l'ensemble du jeu de données (batch)
Les calculs ci-dessus ont été fait pour une donnée d'entrée (pour simplifier les écritures). Dans le cas réel d'un jeu de N données notre fonction coût globale est moyenne des coûts pour chaque donnée :
$$E(W,D) = -\frac{1}{N}\sum_{k=1}^{N}(y_k.log(\hat y_k + \epsilon)+(1-y_k).log(1-\hat y_k +\epsilon))$$
Ce qui nous donne les formules suivantes :
- couche de sortie (en notant $\delta^{(k)} = \hat y^{(k)} - y^{(k)}$) :
$$\Delta w_5 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta^{(k)}.y_1^{(k)}$$
$$\Delta w_6 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta^{(k)}.y_2^{(k)}$$
$$\Delta b = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta^{(k)}$$
- couche cachée (en notant $\delta_1^{(k)} = \delta^{(k)}.w_5. (1 - {y_1^{(k)}}^2)$ et $\delta_2^{(k)} = \delta^{(k)}.w_6. (1 - {y_2^{(k)}}^2)$ ) :
$$\Delta w_1 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta_1^{(k)}.x_1^{(k)}$$
$$\Delta w_3 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta_1^{(k)}.x_2^{(k)}$$
$$\Delta w_2 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta_2^{(k)}.x_1^{(k)}$$
$$\Delta w_4 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta_2^{(k)}.x_2^{(k)}$$
$$\Delta b_1 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta_1^{(k)}$$
$$\Delta b_2 = -\eta.\frac{1}{N} \sum_{k=1}^{N}. \delta_2^{(k)}$$



## 6. Apprentissage

In [ ]:
# TODO : boucle d’apprentissage
lr=0.1
iteMax=30000
pertes=[]

for ite in range(iteMax):
    
    # ----- Forward -----
    # Couche cachée
    ...

    # Couche sortie
    ...  

    # ----- Perte  -----
    ...

    # ----- Backward -----
    ...
   
plt.plot(pertes)
plt.title("Évolution de la perte")
plt.xlabel("Itérations"); plt.ylabel("perte")
plt.tight_layout()
plt.show()


In [ ]:
# Frontière de décision
xx, yy = np.meshgrid(np.linspace(-0.5, 1.5, 200),
                     np.linspace(-0.5, 1.5, 200))
grid = np.c_[xx.ravel(), yy.ravel()]

v1g = w1*grid[:,0] + w3*grid[:,1] + b1
v2g = w2*grid[:,0] + w4*grid[:,1] + b2
y1g = np.tanh(v1g)
y2g = np.tanh(v2g)
vg  = w5*y1g + w6*y2g + b
zz  = sigmoid(vg).reshape(xx.shape)

cf = plt.contourf(xx, yy, zz, levels=50, alpha=0.6)
plt.colorbar(cf, label="p(y=1)")
cs = plt.contour(xx, yy, zz, levels=[0.5], linewidths=2)
plt.clabel(cs, fmt={0.5: 'p=0.5'})
plt.scatter(X[y==0,0], X[y==0,1], label="Classe 0", edgecolors='k')
plt.scatter(X[y==1,0], X[y==1,1], label="Classe 1", edgecolors='k')
plt.xlabel("x1"); plt.ylabel("x2"); plt.title("Frontière de décision MLP 2–2–1")
plt.legend(); plt.tight_layout(); plt.show()